This notebook splits entire recording into tasks and excludes all data not beeing baseline or task.

Required data can be found smb://nas-weber01.unisg.ch/data/Nassy/03_Online_Model/data

Make sure to create the folder splilt

In [5]:
import pandas as pd
from os import walk

pd.set_option('display.max_rows', 1000)

In [6]:

_, _, file_names = next(walk('./data'))

files = [(name.split('_')[0], name) for name in file_names]

files

[('002', '002_10.07.csv'),
 ('003', '003_10.06.csv'),
 ('001', '001_10.09.csv'),
 ('008', '008_10.01.csv'),
 ('007', '007_10.02.csv'),
 ('006', '006_10.03.csv'),
 ('005', '005_10.04.csv'),
 ('004', '004_10.05.csv')]

In [7]:
tasks = [
    ('t1', 't1_gsr', 't1_eclipse (drawing_a_1.java)'),
    ('t2', 't2_gsr', 't2_eclipse (drawing_a_2.java)'),
    ('t3', 't3_gsr', 't3_eclipse (drawing_b_1.java)'),
    ('t4', 't4_gsr', 't4_eclipse (drawing_b_2.java)'),
    ('t5', 't5_gsr', 't5_eclipse (drawing_b_3.java)'),
    ('t6', 't6_gsr', 't6_eclipse (drawing_b_6.java)'),
    ('t7', 't7_gsr', 't7_eclipse (drawing_b_9.java)'),
    ('t8', 't8_gsr', 't8_eclipse (drawing_b_11.java)'),
]

tasks

[('t1', 't1_gsr', 't1_eclipse (drawing_a_1.java)'),
 ('t2', 't2_gsr', 't2_eclipse (drawing_a_2.java)'),
 ('t3', 't3_gsr', 't3_eclipse (drawing_b_1.java)'),
 ('t4', 't4_gsr', 't4_eclipse (drawing_b_2.java)'),
 ('t5', 't5_gsr', 't5_eclipse (drawing_b_3.java)'),
 ('t6', 't6_gsr', 't6_eclipse (drawing_b_6.java)'),
 ('t7', 't7_gsr', 't7_eclipse (drawing_b_9.java)'),
 ('t8', 't8_gsr', 't8_eclipse (drawing_b_11.java)')]

In [8]:



for file_name in files:
    print(file_name)
    df = pd.read_csv(f'./data/{file_name[1]}', comment='#')
    df.set_index(pd.TimedeltaIndex(df['Timestamp'].values, unit="ms"), inplace=True)


    et = df[['SourceStimuliName', 'SlideEvent', 'ET_PupilRight', 'ET_PupilLeft', 'ET_DistanceRight', 'ET_DistanceLeft', 'ET_ValidityLeft', 'ET_ValidityRight', 'Path', 'Line', 'Col', 'ET_GazeLeftx', 'ET_GazeLefty', 'ET_GazeRightx', 'ET_GazeRighty', "Timestamp", "ET_TimeSignal"]]
    stimuli = et[['SourceStimuliName', 'SlideEvent']].fillna('')

    for task in tasks:
        print(task)
        t1_gsr_start_idx = stimuli[stimuli['SlideEvent'] == 'StartMedia']['SourceStimuliName'].eq(task[1]).idxmax()
        t1_gsr_end_idx = stimuli[stimuli['SlideEvent'] == 'EndMedia']['SourceStimuliName'].eq(task[1]).idxmax()
        t1_gsr = et[(et.index > t1_gsr_start_idx) & (et.index < t1_gsr_end_idx)]

        t1_gsr['ET_PupilRight'] = t1_gsr['ET_PupilRight'].replace(-1, pd.np.nan)
        t1_gsr['ET_PupilLeft'] = t1_gsr['ET_PupilLeft'].replace(-1, pd.np.nan)
        t1_gsr['ET_GazeLeftx'] = t1_gsr['ET_GazeLeftx'].replace(-1, pd.np.nan)
        t1_gsr['ET_GazeLefty'] = t1_gsr['ET_GazeLefty'].replace(-1, pd.np.nan)
        t1_gsr['ET_GazeRightx'] = t1_gsr['ET_GazeRightx'].replace(-1, pd.np.nan)
        t1_gsr['ET_GazeRighty'] = t1_gsr['ET_GazeRighty'].replace(-1, pd.np.nan)
        t1_gsr['Timestamp'] = t1_gsr.apply(lambda x: int(x['Timestamp'] * 10000000), axis=1)
        t1_gsr['type'] = 'B'



        t1_eclipse_start_idx = stimuli[stimuli['SlideEvent'] == 'StartMedia']['SourceStimuliName'].eq(task[1]).idxmax()
        t1_eclipse_end_idx = stimuli[stimuli['SlideEvent'] == 'EndMedia']['SourceStimuliName'].eq(task[2]).idxmax()
        t1_eclipse = et[(et.index > t1_eclipse_start_idx) & (et.index < t1_eclipse_end_idx)]
        t1_eclipse['ET_PupilRight'] = t1_eclipse['ET_PupilRight'].replace(-1, pd.np.nan)
        t1_eclipse['ET_PupilLeft'] = t1_eclipse['ET_PupilLeft'].replace(-1, pd.np.nan)
        t1_eclipse['ET_GazeLeftx'] = t1_eclipse['ET_GazeLeftx'].replace(-1, pd.np.nan)
        t1_eclipse['ET_GazeLefty'] = t1_eclipse['ET_GazeLefty'].replace(-1, pd.np.nan)
        t1_eclipse['ET_GazeRightx'] = t1_eclipse['ET_GazeRightx'].replace(-1, pd.np.nan)
        t1_eclipse['ET_GazeRighty'] = t1_eclipse['ET_GazeRighty'].replace(-1, pd.np.nan)
        t1_eclipse['Timestamp'] = t1_eclipse.apply(lambda x: int(x['Timestamp'] * 10000000), axis=1)
        t1_eclipse['type'] = 'M'

        concat = pd.concat([t1_gsr, t1_eclipse])
        concat['subject'] = file_name[0]
        concat['task'] = task[0] 
        
        
        concat.to_csv(f'split/{file_name[0]}_{task[0]}.csv', index=False, header=True)


('002', '002_10.07.csv')
('t1', 't1_gsr', 't1_eclipse (drawing_a_1.java)')


<ipython-input-8-2c53120ed1e3>:16: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  t1_gsr['ET_PupilRight'] = t1_gsr['ET_PupilRight'].replace(-1, pd.np.nan)
<ipython-input-8-2c53120ed1e3>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t1_gsr['ET_PupilRight'] = t1_gsr['ET_PupilRight'].replace(-1, pd.np.nan)
<ipython-input-8-2c53120ed1e3>:17: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  t1_gsr['ET_PupilLeft'] = t1_gsr['ET_PupilLeft'].replace(-1, pd.np.nan)
<ipython-input-8-2c53120ed1e3>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Da

('t2', 't2_gsr', 't2_eclipse (drawing_a_2.java)')
('t3', 't3_gsr', 't3_eclipse (drawing_b_1.java)')
('t4', 't4_gsr', 't4_eclipse (drawing_b_2.java)')
('t5', 't5_gsr', 't5_eclipse (drawing_b_3.java)')
('t6', 't6_gsr', 't6_eclipse (drawing_b_6.java)')
('t7', 't7_gsr', 't7_eclipse (drawing_b_9.java)')
('t8', 't8_gsr', 't8_eclipse (drawing_b_11.java)')
('003', '003_10.06.csv')
('t1', 't1_gsr', 't1_eclipse (drawing_a_1.java)')
('t2', 't2_gsr', 't2_eclipse (drawing_a_2.java)')
('t3', 't3_gsr', 't3_eclipse (drawing_b_1.java)')
('t4', 't4_gsr', 't4_eclipse (drawing_b_2.java)')
('t5', 't5_gsr', 't5_eclipse (drawing_b_3.java)')
('t6', 't6_gsr', 't6_eclipse (drawing_b_6.java)')
('t7', 't7_gsr', 't7_eclipse (drawing_b_9.java)')
('t8', 't8_gsr', 't8_eclipse (drawing_b_11.java)')
('001', '001_10.09.csv')
('t1', 't1_gsr', 't1_eclipse (drawing_a_1.java)')
('t2', 't2_gsr', 't2_eclipse (drawing_a_2.java)')
('t3', 't3_gsr', 't3_eclipse (drawing_b_1.java)')
('t4', 't4_gsr', 't4_eclipse (drawing_b_2.java)'